###### Imports and Settings

In [5]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# Comprehensive Plan Data Pull  

In [6]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [7]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [7]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4702180', #Ashland City: Cheatham
          '1600000US4739660', #Kingston Springs: Cheatham
          '1600000US4757480', #Pegram: Cheatham
          '1600000US4759560', #Pleasant View: Cheatham
          '1600000US4704620', #Belle Meade: Davidson
          '1600000US4705140', #Berry Hill: Davidson
          '1600000US4727020', #Forest Hills: Davidson
          '1600000US4729920', #Goodlettsville: Davidson/Sumner
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4754780', #Oak Hill: Davidson
          '1600000US4763140', #Ridgetop: Davidson/Robertson
          '1600000US4709880', #Burns: Dickson
          '1600000US4713080', #Charlotte: Dickson
          '1600000US4720620', #Dickson: Dickson
          '1600000US4769080', #Slayden: Dickson
          '1600000US4776860', #Vanleer: Dickson
          '1600000US4779980', #White Bluff: Dickson
          '1600000US4724320', #Erin: Houston
          '1600000US4773460', #Tennessee Ridge: Houston/Stewart
          '1600000US4744840', #McEwen: Humphreys
          '1600000US4752820', #New Johnsonville: Humphreys
          '1600000US4778560', #Waverly: Humphreys
          '1600000US4716540', #Columbia: Maury
          '1600000US4751080', #Mount Pleasant: Maury
          '1600000US4770580', #Spring Hill: Maury/Williamson
          '1600000US4715160', #Clarksville: Montgomery
          '1600000US4700200', #Adams: Robertson
          '1600000US4711980', #Cedar Hill: Robertson
          '1600000US4716980', #Coopertown: Robertson
          '1600000US4718420', #Cross Plains: Robertson
          '1600000US4730960', #Greenbrier: Robertson
          '1600000US4748980', #Millersville: Robertson/Sumner
          '1600000US4760280', #Portland: Robertson/Sumner
          '1600000US4770500', #Springfield: Robertson
          '1600000US4780200', #White House: Robertson/Sumner
          '1600000US4722360', #Eagleville: Rutherford
          '1600000US4741200', #La Vergne: Rutherford
          '1600000US4751560', #Murfreesboro: Rutherford
          '1600000US4769420', #Smyrna: Rutherford
          '1600000US4718820', #Cumberland City: Stewart
          '1600000US4721400', #Dover: Stewart
          '1600000US4728540', #Gallatin: Sumner
          '1600000US4733280', #Hendersonville: Sumner
          '1600000US4779420', #Westmoreland: Sumner
          '1600000US4708280', #Brentwood: Williamson
          '1600000US4725440', #Fairview: Williamson
          '1600000US4727740', #Franklin: Williamson
          '1600000US4753460', #Nolensville: Williamson
          '1600000US4773900', #Thompson's Station: Williamson
          '1600000US4741520', #Lebanon: Wilson
          '1600000US4750780', #Mount Juliet: Wilson
          '1600000US4778320', #Watertown: Wilson
          '1600000US4740160', #Lafayette: Macon
          '1600000US2169114', #Scottsville city, Kentucky
          '1600000US2128918', #Franklin city, Kentucky
          '1600000US4749460'] #Mitchelville city: Sumner

## Read In Data Guide

The "head" should never be more than 2 variables, and the "tail" never more than 2. Since we can pull 50 variables at once this means that we can systematically program in 46 variables for each pull, so that's:
+ dg1: ID's 1 - 46  
+ dg2: ID's 47-92  
**This data guide stops at ID 88 as of 7/8/2022.**

### SF1

In [8]:
dataguide = pd.read_csv('../../data guides/DATA GUIDE SF12000.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [9]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
url_str= 'https://api.census.gov/data/2000/dec/sf1?key='+api_key

In [10]:
# ONE 2000
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg1['SF1 Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg1['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
#add stuff to columns list
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
#ky places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
#places call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
# #tract call
# predicates= {}
# get_vars= var_list
# predicates['get'] = ','.join(get_vars)
# predicates['for'] = 'tract:0902'
# predicates['in'] = 'state:47'
# predicates['%20'] = 'county:169'
# data= requests.get(url_str, params= predicates)
# col_names = col_list
# tract=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
# df = df.append(tract).reset_index(drop = True)
#state call
col_list.remove(tail_cols2)
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
#national call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
onepull = df
print('Okay Finished')

Okay Finished


In [11]:
one = onepull

In [12]:
head1 = 'NAME' #TWO list params
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
var_list = list(dg2['SF1 Variable']) #make variables list
var_list = deque(var_list) #add stuff to variables list
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
col_list = list(dg2['Column Name']) #make columns list
col_list.append(tail_cols1) #add stuff to columns list
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
predicates= {} #add stuff to columns list
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = df.append(kycos).reset_index(drop = True)
predicates= {} #places call
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
predicates= {} #ky places call
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)]
df = df.append(places).reset_index(drop = True)
col_list.remove(tail_cols2) #state call
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = df.append(state, ignore_index = True)
predicates= {} #national call
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = df.append(national, ignore_index = True)
twopull = df
print('Okay Finished')

Okay Finished


In [13]:
two = twopull

In [14]:
last = two

## Joining

In [15]:
one = one.drop(columns = ['StateFIPS','GeoFIPS'])
last = last.drop(columns = 'NAME')

In [16]:
data = one.merge(last, on = 'GEO_ID')

In [17]:
data.head()

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_total_oneracealone,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,hhtype_total_series,hhtype_oneperson,hhtype_oneperson_male,hhtype_oneperson_female,hhtype_twoormore,hhtype_twoormore_family,hhtype_twoormore_family_marriedcouplefam,hhtype_twoormore_family_marriedcouplefam_ownchildrenunder18,hhtype_twoormore_family_marriedcouplefam_noownchildrenunder18,hhtype_twoormore_family_otherfam,hhtype_twoormore_family_otherfam_malenospouse,hhtype_twoormore_family_otherfam_malenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_malenospouse_noownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse,hhtype_twoormore_family_otherfam_femalenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse_noownchildrenunder18,hhtype_twoormore_nonfamily,hhtype_twoormore_nonfamily_male,hhtype_twoormore_nonfamily_female,StateFIPS,GeoFIPS
0,"Cheatham County, Tennessee",0500000US47021,35912,35912,17981,1372,1473,1487,829,440,180,177,570,1133,1388,1787,1617,1411,1231,881,284,394,221,268,389,226,135,88,17931,1187,1330,1398,854,369,173,182,518,1201,1487,1780,1649,1372,1161,850,289,373,217,312,433,327,204,265,35912,35660,34783,532,135,63,17,130,252,34506,437,2.76,13508,13508,12878,630,12878,10773,2105,12878,2177,1042,1135,10701,10162,8356,4026,4330,1806,564,357,207,1242,714,528,539,330,209,47,021
1,"Davidson County, Tennessee",0500000US47037,569891,569891,275865,19466,18200,17089,10120,9254,4730,4558,14070,26542,24511,24019,22218,19891,16467,11655,3874,5310,3010,4378,6608,4941,2961,1993,294026,18347,17524,16143,9558,9399,4935,4832,14420,25950,23184,24029,23233,20978,17698,13313,4521,6409,4016,5878,9508,8301,5841,6009,569891,558652,381783,147696,1679,13275,403,13816,11239,371150,26091,2.30,252977,252977,237405,15572,237405,131340,106065,237405,79249,34151,45098,158156,138106,94784,39175,55609,43322,9283,4111,5172,34039,19985,14054,20050,11527,8523,47,037
2,"Dickson County, Tennessee",0500000US47043,43156,43156,21158,1538,1686,1726,957,557,250,236,701,1416,1594,1779,1756,1467,1361,1104,384,548,338,428,538,414,242,138,21998,1437,1584,1601,957,532,246,227,744,1459,1649,1753,1828,1513,1431,1129,381,556,322,505,693,579,462,410,43156,42718,40243,1978,172,116,5,204,438,40020,484,2.59,17614,17614,16473,1141,16473,12539,3934,16473,3678,1591,2087,12795,12175,9604,4322,5282,2571,670,387,283,1901,1162,739,620,384,236,47,043
3,"Houston County, Tennessee",0500000US47083,8088,8088,3999,268,311,266,174,86,36,39,146,264,256,255,300,280,266,234,83,150,73,112,158,112,77,53,4089,273,243,298,137,78,37,43,127,244,240,277,271,293,274,260,92,137,92,112,180,151,113,117,8088,8011,7650,268,15,10,5,63,77,7611,101,2.46,3901,3901,3216,685,3216,2476,740,3216,815,368,447,2401,2300,1833,746,1087,467,134,59,75,333,194,139,101,68,33,47,083
4,"Humphreys County, Tennessee",0500000US47085,17929,17929,8819,567,610,660,402,214,108,110,280,537,536,651,701,635,624,589,202,275,153,240,312,225,114,74,9110,505,597,584,357,198,

In [18]:
data = data.set_index(['NAME', 'GEO_ID'])

In [19]:
transp = data.transpose()
transp.head()

NAME,"Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Macon County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Allen County, Kentucky","Simpson County, Kentucky","Franklin city, Kentucky","Scottsville city, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee","Smyrna town, Tennessee","Springfield city, Tennessee","Spring Hill city, Tennessee","Tennessee Ridge town, Tennessee","Thompson's Station town, Tennessee","Vanleer town, Tennessee","Watertown city, Tennessee","Waverly city, Tennessee","Westmoreland town, Tennessee","White Bluff town, Tennessee","White House city, Tennessee",Tennessee,United States
GEO_ID,0500000US47021,0500000US47037,0500000US47043,0500000US47083,0500000US47085,0500000US47111,0500000US47119,0500000US47125,0500000US47147,0500000US47149,0500000US47161,0500000US47165,0500000US47169,0500000US47187,0500000US47189,0500000US21003,0500000US21213,1600000US2128918,1600000US2169114,1600000US4700200,1600000US4702180,1600000US4704620,1600000US4705140,1600000US4708280,1600000US4709880,1600000US4711980,1600000US4713080,1600000US4715160,1600000US4716540,1600000US4716980,1600000US4718420,1600000US4718820,1600000US4720620,1600000US4721400,1600000US4722360,1600000US4724320,1600000US4725440,1600000US4727020,1600000US4727740,1600000US4728540,1600000US4729920,1600000US4730960,1600000US4733280,1600000US4739660,1600000US4740160,1600000US4741200,1600000US4741520,1600000US4744840,1600000US4748980,1600000US4749460,1600000US4750780,1600000US4751080,1600000US4751560,1600000US4752006,1600000US4752820,1600000US4753460,1600000US4754780,1600000US4757480,1600000US4759560,1600000US4760280,1600000US4763140,1600000US4769080,1600000US4769420,1600000US4770500,1600000US4770580,1600000US4773460,1600000US4773900,1600000US4776860,1600000US4778320,1600000US4778560,1600000US4779420,1600000US4779980,1600000US4780200,0400000US47,0100000US
pop,35912,569891,43156,8088,17929,20386,69498,134768,54433,182023,12370,130449,7259,126638,88809,17800,16405,7996,4327,566,3641,2943,674,23445,1366,298,1153,103455,33055,3027,1381,316,12244,1442,464,1490,5800,4710,41842,23230,13780,4940,40620,2773,3885,18687,20235,1702,5308,207,12366,4491,68816,545524,1905,3099,4493,2146,2934,8458,1083,185,25569,14329,7715,1334,1283,310,1358,4028,2093,2142,7220,5689283,281421906
agebysex_total_series,35912,569891,43156,8088,17929,20386,69498,134768,54433,182023,12370,130449,7259,126638,88809,17800,16405,7996,4327,566,3641,2943,674,23445,1366,298,1153,103455,33055,3027,1381,31

In [20]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [21]:
data = transp

In [22]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [23]:
data = data.transpose().reset_index()

In [24]:
data.head(30)

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_total_oneracealone,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,hhtype_total_series,hhtype_oneperson,hhtype_oneperson_male,hhtype_oneperson_female,hhtype_twoormore,hhtype_twoormore_family,hhtype_twoormore_family_marriedcouplefam,hhtype_twoormore_family_marriedcouplefam_ownchildrenunder18,hhtype_twoormore_family_marriedcouplefam_noownchildrenunder18,hhtype_twoormore_family_otherfam,hhtype_twoormore_family_otherfam_malenospouse,hhtype_twoormore_family_otherfam_malenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_malenospouse_noownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse,hhtype_twoormore_family_otherfam_femalenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse_noownchildrenunder18,hhtype_twoormore_nonfamily,hhtype_twoormore_nonfamily_male,hhtype_twoormore_nonfamily_female,StateFIPS,GeoFIPS
0,"Cheatham County, Tennessee",0500000US47021,35912.0,35912.0,17981.0,1372.0,1473.0,1487.0,829.0,440.0,180.0,177.0,570.0,1133.0,1388.0,1787.0,1617.0,1411.0,1231.0,881.0,284.0,394.0,221.0,268.0,389.0,226.0,135.0,88.0,17931.0,1187.0,1330.0,1398.0,854.0,369.0,173.0,182.0,518.0,1201.0,1487.0,1780.0,1649.0,1372.0,1161.0,850.0,289.0,373.0,217.0,312.0,433.0,327.0,204.0,265.0,35912.0,35660.0,34783.0,532.0,135.0,63.0,17.0,130.0,252.0,34506.0,437.0,2.76,13508.0,13508.0,12878.0,630.0,12878.0,10773.0,2105.0,12878.0,2177.0,1042.0,1135.0,10701.0,10162.0,8356.0,4026.0,4330.0,1806.0,564.0,357.0,207.0,1242.0,714.0,528.0,539.0,330.0,209.0,47.0,21.0
1,"Davidson County, Tennessee",0500000US47037,569891.0,569891.0,275865.0,19466.0,18200.0,17089.0,10120.0,9254.0,4730.0,4558.0,14070.0,26542.0,24511.0,24019.0,22218.0,19891.0,16467.0,11655.0,3874.0,5310.0,3010.0,4378.0,6608.0,4941.0,2961.0,1993.0,294026.0,18347.0,17524.0,16143.0,9558.0,9399.0,4935.0,4832.0,14420.0,25950.0,23184.0,24029.0,23233.0,20978.0,17698.0,13313.0,4521.0,6409.0,4016.0,5878.0,9508.0,8301.0,5841.0,6009.0,569891.0,558652.0,381783.0,147696.0,1679.0,13275.0,403.0,13816.0,11239.0,371150.0,26091.0,2.30,252977.0,252977.0,237405.0,15572.0,237405.0,131340.0,106065.0,237405.0,79249.0,34151.0,45098.0,158156.0,138106.0,94784.0,39175.0,55609.0,43322.0,9283.0,4111.0,5172.0,34039.0,19985.0,14054.0,20050.0,11527.0,8523.0,47.0,37.0
2,"Dickson County, Tennessee",0500000US47043,43156.0,43156.0,21158.0,1538.0,1686.0,1726.0,957.0,557.0,250.0,236.0,701.0,1416.0,1594.0,1779.0,1756.0,1467.0,1361.0,1104.0,384.0,548.0,338.0,428.0,538.0,414.0,242.0,138.0,21998.0,1437.0,1584.0,1601.0,957.0,532.0,246.0,227.0,744.0,1459.0,1649.0,1753.0,1828.0,1513.0,1431.0,1129.0,381.0,556.0,322.0,505.0,693.0,579.0,462.0,410.0,43156.0,42718.0,40243.0,1978.0,172.0,116.0,5.0,204.0,438.0,40020.0,484.0,2.59,17614.0,17614.0,16473.0,1141.0,16473.0,12539.0,3934.0,16473.0,3678.0,1591.0,2087.0,12795.0,12175.0,9604.0,4322.0,5282.0,2571.0,670.0,387.0,283.0,1901.0,1162.0,739.0,620.0,384.0,236.0,47.0,43.0
3,"Houston County, Tennessee",0500000US47083,8088.

In [25]:
data.to_csv('../../data/SF12000.csv', index = False)